In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType

In [2]:
spark = SparkSession.builder.appName("jupyter_test") \
    .config("spark.master", "yarn") \
    .config("spark.jars",   "/opt/spark/jars/postgresql-42.6.0.jar") \
    .config("spark.eventLog.enabled",          "true") \
    .config("spark.eventLog.dir",              "file:///opt/spark/eventLog") \
    .config("spark.history.fs.logDirectory",   "file:///opt/spark/eventLog") \
    .config("spark.hadoop.yarn.timeline-service.enabled", "false") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d76fd9f9-1ef8-4776-93d8-43df040cfdd2;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.0 in central
:: resolution report :: resolve 96ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

In [16]:
csvDf = spark.read \
    .option("header", True) \
    .csv("/games.csv", inferSchema =True)

In [17]:
csvDf.show()

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
|app_id|               title|date_release| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
| 13500|Prince of Persia:...|  2008-11-21|true|false|false|  Very Positive|            84|        2199|       9.99|          9.99|     0.0|      true|
| 22364|BRINK: Agents of ...|  2011-08-03|true|false|false|       Positive|            85|          21|       2.99|          2.99|     0.0|      true|
|113020|Monaco: What's Yo...|  2013-04-24|true| true| true|  Very Positive|            92|        3722|      14.99|         14.99|     0.0|      true|
|226560|  Escape Dead Island|  2014-11-18|true|false|false|          Mixed|            61|    

In [21]:
csvDf.write.format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "testuser.games2") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .save()

In [22]:
tableDf = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "testuser.games2") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

In [23]:
tableDf.show()

+-------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+
| app_id|               title|date_release| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+-------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+
| 410900|               Forts|  2017-04-19|true|false|false|       Very Positive|            92|       13063|      14.99|         14.99|     0.0|      true|
| 411300|                ELEX|  2017-10-17|true|false|false|     Mostly Positive|            72|       11330|       7.49|         29.99|    75.0|      true|
| 411890|   DCS: Persian Gulf|  2018-06-01|true|false|false|       Very Positive|            82|         107|      49.99|         49.99|     0.0|      true|
| 411893| DCS: F-14A/B Tomcat|  2019-03-13|true|false|fals